In [46]:
import os
import random
import time
from datetime import datetime
import csv
import pandas as pd
from sklearn.linear_model import LinearRegression

# Create output directory
output_dir = "sensor_data/csv"
os.makedirs(output_dir, exist_ok=True)


In [47]:
def classify_health_environment(voc, nh3, h2s, no2):
    score = 0
      # NH₃ (Ammoniak) – z.b Uringeruch
    if nh3 is not None and nh3 > 5:
        score += 1
    # H₂S (Schwefelwasserstoff) – stinkendes Gas
    if h2s is not None and h2s > 0.005:
        score += 0
    # VOC (flüchtige organische Verbindungen) – allgemeine Geruchsquelle
    if voc is not None and voc > 450:
        score += 0
    # NO₂ (Stickstoffdioxid) – Schlechte Belüftung
    if no2 is not None and no2 > 250:     #300 Kommt nie vor, no2 hat kein geruch, lasse es dennoch drinnen für was anderes
        score += 0

    if score == 0:
        return "Fresh"
    elif score == 1:
        return "Noticeable"
    else:
        return "Stinky"

In [48]:
# Function to generate random air quality data
def generate_air_quality_data(sensor_id=3):
    nh3 = round(random.uniform(0, 10), 2)
    h2s = round(random.uniform(0, 0.01), 4)
    voc = round(random.uniform(50, 600), 2)
    no2 = round(random.uniform(0, 300), 2) if random.random() > 0.2 else None   #damit ein Wert mal fehlt 

    health_environment = classify_health_environment(nh3, h2s, voc, no2)

    return {
        "sensor_id": sensor_id,
        "timestamp": datetime.now().isoformat(),
        "nh3": nh3,
        "h2s": h2s,
        "voc": voc,
        "no2": no2,
        "health_environment": health_environment.lower()
    }

In [49]:
# Impute only no2 values using mean, median, mode or regression
def impute_missing_values(df, strategy='mean'):
    if strategy == 'mean':
        df['no2'] = df['no2'].fillna(df['no2'].mean()).round(2)
    elif strategy == 'median':
         df['no2'] = df['no2'].fillna(df['no2'].median()).round(2)
    elif strategy == 'mode':
        df['no2'] = df['no2'].fillna(df['no2'].mode()).round(2)
    #elif strategy == 'regression':             Haben keinen direkten Zusammenhang zwischen den Werten weshalb des net funktioniert
        # Only impute no2 values using so2 as predictor
    #    hum_train = df.dropna(subset=['no2'])
    #    model = LinearRegression()
    #    model.fit(hum_train[['so2']], hum_train['no2'])
    #    hum_missing = df[df['no2'].isnull()]
    #    df.loc[df['no2'].isnull(), 'no2'] = model.predict(hum_missing[['so2']]).round(2)
    return df

Daten in CSV Speichern

In [ ]:
# Function to save data as CSV
def save_air_quality_csv(filename, data):
    if not data:
        return
    filepath = os.path.join(output_dir, filename)
    with open(filepath, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"Saved Air Quality CSV: {filepath}")

# User input for number of times to run
num_iterations = int(input("Enter number of air quality readings to generate: "))

# Generate multiple batches of air quality data
readings = []
for i in range(num_iterations):
    air_quality_data = generate_air_quality_data()
    print(air_quality_data)
    readings.append(air_quality_data)
    time.sleep(1)  # Simulate real-time data generation

    # Add a duplicate at every 3rd reading
    if i % 3 == 0 and i != 0:
        readings.append(air_quality_data)
        print(air_quality_data)

# Create DataFrame
df = pd.DataFrame(readings)

# Save raw data (with duplicates)
save_air_quality_csv("air_quality_readings_errors.csv", df.to_dict(orient='records'))

{'sensor_id': 3, 'timestamp': '2025-06-02T11:58:44.222903', 'nh3': 0.59, 'h2s': 0.0015, 'voc': 244.54, 'no2': 43.03, 'health_environment': 'fresh'}
{'sensor_id': 3, 'timestamp': '2025-06-02T11:58:44.224856', 'nh3': 2.48, 'h2s': 0.0045, 'voc': 479.2, 'no2': 188.52, 'health_environment': 'fresh'}
{'sensor_id': 3, 'timestamp': '2025-06-02T11:58:44.226614', 'nh3': 7.99, 'h2s': 0.0034, 'voc': 159.96, 'no2': 256.38, 'health_environment': 'fresh'}
{'sensor_id': 3, 'timestamp': '2025-06-02T11:58:44.228322', 'nh3': 6.61, 'h2s': 0.0093, 'voc': 271.57, 'no2': 147.97, 'health_environment': 'fresh'}
{'sensor_id': 3, 'timestamp': '2025-06-02T11:58:44.228322', 'nh3': 6.61, 'h2s': 0.0093, 'voc': 271.57, 'no2': 147.97, 'health_environment': 'fresh'}
{'sensor_id': 3, 'timestamp': '2025-06-02T11:58:44.230137', 'nh3': 7.0, 'h2s': 0.0033, 'voc': 510.12, 'no2': 282.9, 'health_environment': 'fresh'}
{'sensor_id': 3, 'timestamp': '2025-06-02T11:58:44.231775', 'nh3': 9.83, 'h2s': 0.0018, 'voc': 556.88, 'no2': 

Doppelte Werte entfernen und Fehlende Ergänzen:

In [51]:
# Remove duplicated rows
df = df.drop_duplicates()

# Impute missing no2 values 
df = impute_missing_values(df, strategy="mean")  #besser, aber anfälliger für ausreißer

# Save cleaned data (without duplicates)
save_air_quality_csv("air_quality_readings.csv", df.to_dict(orient='records'))

#df.to_csv("sensor_data/csv/air_quality_reading.csv", index=False)

Saved Air Quality CSV: sensor_data/csv\air_quality_readings.csv
